In [1]:
import os
import google.generativeai as genai

In [2]:
try:
    api_key = os.environ.get("GEMINI_API_KEY")
    if not api_key:
        raise ValueError(
            "API key not found. Please set the GOOGLE_API_KEY environment variable."
        )
    genai.configure(api_key=api_key)
except Exception as e:
    print(f"Error configuring the API: {e}")
    print("Please make sure you have set the GOOGLE_API_KEY environment variable.")
    exit()

In [3]:
class UILanguageAgent:
    """
    Handles the static UI text translations for the application.
    """

    def __init__(self):
        self.translations = {
            "welcome": "Chào mừng bạn đến với gia sư hình học AI! Hãy bắt đầu với bài toán đầu tiên.",
            "generating_problem": "Đang tạo một bài toán mới...",
            "problem": "Đây là đề bài:",
            "current_question": "Câu hỏi hiện tại:",
            "your_answer_prompt": "Hãy suy nghĩ và nhập câu trả lời của bạn (nhập 'hint' để xem gợi ý, Enter để xem lời giải): ",
            "generating_hints": "Đang tạo gợi ý...",
            "generating_solution": "Đang tạo lời giải chi tiết...",
            "feedback_start": "Cảm ơn bạn. Dưới đây là lời giải chi tiết:",
            "question_solution": "Lời giải cho câu hỏi này:",
            "next_question": "Chuyển sang câu hỏi tiếp theo...",
            "all_questions_completed": "Bạn đã hoàn thành tất cả các câu hỏi!",
            "next_problem_prompt": "Bạn có muốn thử một bài toán khác không? (yes/no): ",
            "goodbye": "Tạm biệt! Chúc bạn học tốt.",
            "error_problem": "Rất tiếc, đã có lỗi khi tạo bài toán. Vui lòng thử lại.",
            "error_solution": "Rất tiếc, đã có lỗi khi tạo lời giải. Vui lòng thử lại.",
            "error_hints": "Rất tiếc, đã có lỗi khi tạo gợi ý. Vui lòng thử lại.",
        }

    def get_text(self, key):
        return self.translations.get(key, "Translation not found.")

In [4]:
class ProblemGeneratorAgent:
    """
    Uses an LLM to generate geometry problems dynamically.
    """

    def __init__(self, model):
        self.model = model
        self.prompt = (
            "Hãy tạo một bài toán hình học phẳng ở cấp độ kỳ thi tuyển sinh vào lớp 10 ở Việt Nam. "
            "Bài toán bao gồm 3 câu hỏi, ký hiệu lần lượt là a., b., c. liên quan đến các chủ đề như: "
            "chứng minh tứ giác nội tiếp, tính chất tiếp tuyến, hệ thức lượng trong tam giác vuông, "
            "góc với đường tròn, và chứng minh các điểm thẳng hàng hoặc các đường thẳng song song/vuông góc. "
            "Chỉ cung cấp đề bài, không cung cấp lời giải."
        )

    def get_problem(self):
        """Generates and returns a problem statement string."""
        try:
            response = self.model.generate_content(self.prompt)
            return response.text
        except Exception as e:
            print(f"An error occurred in ProblemGeneratorAgent: {e}")
            return None

In [5]:
problem = ProblemGeneratorAgent(genai.GenerativeModel("gemini-1.5-pro")).get_problem()
problem.split("\n\n")

['Cho tam giác ABC nhọn (AB < AC) nội tiếp đường tròn (O). Đường cao AD và BE của tam giác ABC cắt nhau tại H. Đường thẳng EF vuông góc với OA cắt AB, AC lần lượt tại M và N. Tiếp tuyến tại B của (O) cắt AO kéo dài tại K.',
 'a. Chứng minh tứ giác BHEK nội tiếp.',
 'b. Chứng minh MN song song với BC và $\\frac{AM}{AB} = \\frac{AN}{AC}$.',
 'c. Gọi I là trung điểm BC. Chứng minh ba điểm H, I, K thẳng hàng.\n']

In [6]:
class MathExpertAgent:
    """
    Uses an LLM to generate solutions for given geometry problems.
    """

    def __init__(self, model):
        self.model = model

    def get_solution(self, problem_statement):
        """Takes a problem statement and returns a detailed solution string."""
        prompt = (
            f"Hãy cung cấp một lời giải chi tiết, từng bước cho bài toán hình học sau đây. "
            f"Trình bày rõ ràng các luận điểm và các định lý được sử dụng. "
            f"Đề bài: {problem_statement}"
        )
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(f"An error occurred in MathExpertAgent: {e}")
            return None

    def get_hints(self, problem_context, question):
        """Generate 3 hints for a specific question within a problem."""
        prompt = (
            f"Cho bài toán hình học sau: {problem_context}\n\n"
            f"Với câu hỏi cụ thể: {question}\n\n"
            f"Hãy tạo ra 3 gợi ý (hints) ngắn gọn để hướng dẫn học sinh giải câu hỏi này. "
            f"Mỗi gợi ý nên được đánh số và viết trên một dòng riêng. "
            f"Gợi ý 1 nên tổng quát nhất, gợi ý 2 cụ thể hơn, gợi ý 3 gần như chỉ ra hướng giải cụ thể."
        )
        try:
            response = self.model.generate_content(prompt)
            hints_text = response.text.strip()
            # Split hints by numbered lines
            hints = [
                line.strip()
                for line in hints_text.split("\n")
                if line.strip() and any(char.isdigit() for char in line[:5])
            ]
            return hints[:3]  # Ensure we only return 3 hints
        except Exception as e:
            print(f"An error occurred in MathExpertAgent while generating hints: {e}")
            return ["Gợi ý không khả dụng do lỗi kỹ thuật."]

    def get_question_solution(self, problem_context, question):
        """Generate solution for a specific question within a problem."""
        prompt = (
            f"Cho bài toán hình học sau: {problem_context}\n\n"
            f"Hãy cung cấp lời giải chi tiết, từng bước cho câu hỏi cụ thể: {question}\n\n"
            f"Trình bày rõ ràng các luận điểm và các định lý được sử dụng."
        )
        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            print(
                f"An error occurred in MathExpertAgent while generating question solution: {e}"
            )
            return None

In [7]:
class TutorAgent:
    """
    The main agent that interacts with the user and orchestrates the other agents.
    """

    def __init__(self):
        self.ui_agent = UILanguageAgent()
        try:
            # Initialize the generative model
            self.model = genai.GenerativeModel("models/gemini-2.5-flash")
            self.problem_generator = ProblemGeneratorAgent(self.model)
            self.math_expert = MathExpertAgent(self.model)
        except Exception as e:
            print(f"Failed to initialize the generative model: {e}")
            self.model = None
        self.problem = None
        self.questions = []

    def start_session(self):
        if not self.model:
            print("Could not start session due to model initialization failure.")
            return

        print(self.ui_agent.get_text("welcome"))
        self.run_tutoring_loop()

    def extract_questions(self, problem_text):
        """Extract individual questions from the problem text."""
        lines = problem_text.split("\n")
        questions = []

        for line in lines:
            line = line.strip()
            if line.startswith(("a.", "b.", "c.", "a)", "b)", "c)")):
                questions.append(line)

        # If no lettered questions found, try to get last 3 non-empty lines
        if not questions:
            non_empty_lines = [line.strip() for line in lines if line.strip()]
            questions = (
                non_empty_lines[-3:] if len(non_empty_lines) >= 3 else non_empty_lines
            )

        return questions

    def handle_question_with_hints(self, question_index, question):
        """Handle a single question with hint system."""
        print(f"\n--------------------------------------------------")
        print(f"{self.ui_agent.get_text('current_question')} {question_index + 1}")
        print(question)
        print("--------------------------------------------------\n")

        # Generate hints for this question
        print(self.ui_agent.get_text("generating_hints"))
        hints = self.math_expert.get_hints(self.problem, question)
        hint_count = 0

        while True:
            user_input = (
                input(self.ui_agent.get_text("your_answer_prompt")).strip().lower()
            )

            if user_input == "hint":
                if hint_count < len(hints):
                    print(f"\nGợi ý {hint_count + 1}: {hints[hint_count]}\n")
                    hint_count += 1
                else:
                    # All hints used, show solution
                    print(f"\n{self.ui_agent.get_text('generating_solution')}")
                    solution = self.math_expert.get_question_solution(
                        self.problem, question
                    )

                    if solution:
                        print(f"\n{self.ui_agent.get_text('question_solution')}")
                        print(solution)
                    else:
                        print(self.ui_agent.get_text("error_solution"))

                    break
            else:
                # User provided an answer or pressed Enter, show solution
                print(f"\n{self.ui_agent.get_text('generating_solution')}")
                solution = self.math_expert.get_question_solution(
                    self.problem, question
                )

                if solution:
                    print(f"\n{self.ui_agent.get_text('question_solution')}")
                    print(solution)
                else:
                    print(self.ui_agent.get_text("error_solution"))

                break

    def run_tutoring_loop(self):
        while True:
            print(f"\n{self.ui_agent.get_text('generating_problem')}")

            # Generate a new problem
            self.problem = self.problem_generator.get_problem()

            if not self.problem:
                print(self.ui_agent.get_text("error_problem"))
                break

            # Display the problem
            print(f"\n--------------------------------------------------")
            print(self.ui_agent.get_text("problem"))
            print(self.problem)
            print("--------------------------------------------------\n")

            # Extract questions from the problem
            self.questions = self.extract_questions(self.problem)

            if not self.questions:
                print(
                    "Không thể phân tích các câu hỏi từ bài toán. Tạo bài toán mới..."
                )
                continue

            # Handle each question individually
            for i, question in enumerate(self.questions):
                self.handle_question_with_hints(i, question)

                if i < len(self.questions) - 1:
                    print(f"\n{self.ui_agent.get_text('next_question')}")

            print(f"\n{self.ui_agent.get_text('all_questions_completed')}")

            # Ask if user wants another problem
            while True:
                user_choice = (
                    input(self.ui_agent.get_text("next_problem_prompt")).lower().strip()
                )
                if user_choice in ["yes", "no", "y", "n"]:
                    break
                else:
                    print("Lựa chọn không hợp lệ. Vui lòng nhập 'yes' hoặc 'no'.")

            if user_choice in ["no", "n"]:
                print(self.ui_agent.get_text("goodbye"))
                break

In [8]:
tutor_v2 = TutorAgent()

In [11]:
tutor_v2.problem = problem

In [12]:
# Test the question extraction functionality
if tutor_v2.problem:
    questions = tutor_v2.extract_questions(tutor_v2.problem)
    print("Extracted questions:")
    for i, q in enumerate(questions):
        print(f"{i+1}. {q}")
else:
    print("No problem available to test question extraction")

Extracted questions:
1. a. Chứng minh tứ giác BHEK nội tiếp.
2. b. Chứng minh MN song song với BC và $\frac{AM}{AB} = \frac{AN}{AC}$.
3. c. Gọi I là trung điểm BC. Chứng minh ba điểm H, I, K thẳng hàng.


In [14]:
tutor_v2.start_session()

Chào mừng bạn đến với gia sư hình học AI! Hãy bắt đầu với bài toán đầu tiên.

Đang tạo một bài toán mới...

--------------------------------------------------
Đây là đề bài:
Dưới đây là một bài toán hình học phẳng được thiết kế theo yêu cầu:

---

**BÀI TOÁN HÌNH HỌC**

Cho đường tròn $(O)$ và một điểm $A$ nằm ngoài đường tròn. Từ $A$ kẻ tiếp tuyến $AB$ đến đường tròn $(O)$ ($B$ là tiếp điểm). Kẻ cát tuyến $ADC$ đến đường tròn $(O)$ sao cho $D$ nằm giữa $A$ và $C$. Gọi $M$ là trung điểm của đoạn thẳng $CD$.

a. Chứng minh rằng bốn điểm $A, B, O, M$ cùng thuộc một đường tròn.
b. Tia $BD$ cắt tia $CM$ tại $K$. (Lưu ý: CM là đường thẳng đi qua C và M, hay chính là đường thẳng AC).
Chứng minh $BD \cdot BK = BC^2$.
c. Gọi $I$ là giao điểm của $BD$ và tiếp tuyến tại $C$ của đường tròn $(O)$. Chứng minh rằng $AC \cdot AD = AB^2$ và bốn điểm $A, C, I, B$ cùng thuộc một đường tròn.

---
--------------------------------------------------


--------------------------------------------------
Câu h

KeyboardInterrupt: 